In [60]:
### CREATING A CLASS FOR STORING DATA FOR EACH NODE ###

class tree():
            
    def __init__(self, branch_len=None, seq_leaf=None, node_name=None, parent=None, child=None, nuc_list=None, lkl_list=None, type=None):
        self.seq_leaf = seq_leaf
        self.child = child ## this is the same as node name
        self.parent = parent
        self.branch_length = branch_len
        self.nuc_list = nuc_list
        self.lkl_list = lkl_list
        self.name = node_name
        self.type = type ## is it the external node or internal node

    def __len__(self):
        return len(self.seq_leaf)
    
    def getSequence(self):
        return self.seq_leaf
    
    def getBranchLength(self):
        return self.branch_length
       
    def getChildrenNodes(self):
        return self.child
    
    def get_nuc_list(self):
        return self.nuc_list
    
    def get_lkl_list(self):
        return self.lkl_list
    
    def initialize_nuc_list(self):
        if self.seq_leaf:
            bases={"A":[1,0,0,0],
                "C":[0,1,0,0],
                "G":[0,0,1,0],
                "T":[0,0,0,1]}
            self.nuc_list=list()
            for base in self.seq_leaf:
                self.nuc_list.append(bases[base])
            return self.nuc_list
        else: return None

In [61]:
### reading data and adding nodes as objects ###

branch_lengths_path="data/branchlength.dat"
tree_path="data/table.dat"
msa_path="data/msa.dat"


### Reading tree topology, branch lengths and seqeunces ###
with open(tree_path,"r") as tree_file, open(branch_lengths_path,"r") as BL_file, open(msa_path,"r") as msa_file:
    node_added_tmp=[] #list with nodes added to tree to update children
    Tree_Data=[]
    line_num=0
    
    bl=BL_file.readline().split(",")
    seq_for_leaves = {}
    for line in msa_file:
       (key, val) = line.split()
       seq_for_leaves[int(key)] = val


    for parent_child in tree_file:
        tmp_list=parent_child.split(",")

        parent_node_name=tmp_list[0]
        child_node_name=tmp_list[1].rstrip()

        branch_len=bl[line_num]

        if int(child_node_name) in seq_for_leaves.keys():
            leaf_seq=seq_for_leaves[int(child_node_name)]
            type="external" ## can be removed
        else:
            leaf_seq=None
            type="internal" # can be removed
        
        new_node = tree(branch_len=branch_len,
            seq_leaf=leaf_seq,
            type=type,
            node_name=child_node_name,
            parent=parent_node_name,
            child=child_node_name,
            nuc_list=[],
            lkl_list=[])
        new_node.initialize_nuc_list()
        
        Tree_Data.append(new_node)
        line_num+=1

In [62]:
## checking if data loaded correctly
for obj in Tree_Data:
    print("node type    : " , obj.type)
    print("Parent name  : ",obj.parent)
    print("current name : ",obj.name)
    print("Child name   : ",obj.child) ## same as node name
    print("Branch length: ",obj.branch_length)
    print("sequence     : ", obj.seq_leaf)
    print("nuc_list updated: " , obj.get_nuc_list())
    print("lkl_list updated: " , obj.get_lkl_list())


node type    :  external
Parent name  :  9
current name :  1
Child name   :  1
Branch length:  0.1
sequence     :  AGATCAAGATCAAGATCAAGATCAAGATCA
nuc_list updated:  [[1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0]]
lkl_list updated:  []
node type    :  external
Parent name  :  9
current name :  2
Child name   :  2
Branch length:  0.4
sequence     :  AGCTCAAGCTCAAGCTCAAGCTCAAGCTCA
nuc_list updated:  [[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0

In [63]:
import numpy as np
import scipy
from scipy.linalg import expm, sinm, cosm
import math
mu=1 # USER INPUT
Q=np.array([[-3*mu, mu, mu, mu], [mu, -3*mu,  mu, mu], [ mu, mu, -3*mu, mu], [ mu, mu, mu, -3*mu]]) #symbolic of the model of evolution


## dict to store liklihoods of nucleotides at each site s in all nodes
lkl_dict = {}


## updating lkl lists for external nodes
for node in Tree_Data:
    if node.getSequence() is not None: # if it is not internal node 
        nuc_list=node.get_nuc_list() # which nuc is present at each pposition --> a list of lists like [[1,0,0,0],[0,1,0,0]....]
        #Probability_Matrix = scipy.linalg.expm(Q * float(node.getBranchLength())) # same for all positions
        #for pos_list in nuc_list:
        #    node.lkl_list.append(np.matmul(Probability_Matrix,pos_list))
        for pos_list in nuc_list:
            node.lkl_list.append(pos_list)
        lkl_dict[node.name]=node.lkl_list


#!! checkpoint
for node in lkl_dict:
    print("dict", node ," ",lkl_dict[node])


dict 1   [[1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0]]
dict 2   [[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0]]
dict 3   [[0, 1, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 

In [64]:
### MAIN CALCULATION ###

## keeping track of parents nodes that ahve been calculated
parent_nodes_visited=[]

for childA in Tree_Data:
    for childB in Tree_Data:
        if childA != childB: ## if not the same node
            # get two siblings from the tree i.e nodes having same parent
            if childA.parent == childB.parent and not (childA.parent in parent_nodes_visited):

                childA_lkl = lkl_dict[childA.name] # we will get a list of lists with liklihood for each nuclleotide at each position
                childB_lkl = lkl_dict[childB.name]
                anc_AB_lkl=[]
                anc_name=childA.parent #could likewise be childB.parent

                # calculating the liklihood of all 4 nucleotides at each position for the parent
                for pos_list in range(len(childA_lkl)): #using all positions in both children nodes, calculate ancetral nucleotide liklihoods
                    P1 = scipy.linalg.expm(Q * float(childA.getBranchLength()))  
                    P2 = scipy.linalg.expm(Q * float(childB.getBranchLength()))  
                    vec1=np.matmul(childA_lkl[pos_list],P1)
                    vec2=np.matmul(childB_lkl[pos_list],P2)
                    vec_anc = [vec1[i] * vec2[i] for i in range(4)]


                    ## !! CHECKPOINT !! ##
                    print("node",childA.parent,vec_anc) ## we are geiting 4 values/pos i.e lkl of all 4 nucleotides at each position.

                    anc_AB_lkl.append(np.array(vec_anc)) ## position lkl to main list for this node

                lkl_dict[anc_name]=anc_AB_lkl # updating lkl_dict
                parent_nodes_visited.append(childA.parent) # mark parent as vivited

node 9 [0.3021667025762747, 0.016444919950394883, 0.016444919950394876, 0.016444919950394876]
node 9 [0.016444919950394876, 0.016444919950394876, 0.30216670257627487, 0.016444919950394873]
node 9 [0.15019111065015153, 0.03308522863990555, 0.016444919950394873, 0.01644491995039488]
node 9 [0.01644491995039488, 0.016444919950394876, 0.016444919950394873, 0.302166702576275]
node 9 [0.01644491995039487, 0.3021667025762748, 0.016444919950394866, 0.016444919950394873]
node 9 [0.3021667025762747, 0.016444919950394883, 0.016444919950394876, 0.016444919950394876]
node 9 [0.3021667025762747, 0.016444919950394883, 0.016444919950394876, 0.016444919950394876]
node 9 [0.016444919950394876, 0.016444919950394876, 0.30216670257627487, 0.016444919950394873]
node 9 [0.15019111065015153, 0.03308522863990555, 0.016444919950394873, 0.01644491995039488]
node 9 [0.01644491995039488, 0.016444919950394876, 0.016444919950394873, 0.302166702576275]
node 9 [0.01644491995039487, 0.3021667025762748, 0.01644491995039

In [65]:
### CHECKPOINT ###
np.log(np.matmul(vec_anc, [0.25,0.25,0.25,0.25]))

-7.352796041811421

In [66]:
lkl_dict

{'1': [[1, 0, 0, 0],
  [0, 0, 1, 0],
  [1, 0, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 1, 0],
  [1, 0, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 1, 0],
  [1, 0, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 1, 0],
  [1, 0, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 1, 0],
  [1, 0, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0]],
 '2': [[1, 0, 0, 0],
  [0, 0, 1, 0],
  [0, 1, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 1, 0],
  [0, 1, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 1, 0],
  [0, 1, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 1, 0],
  [0, 1, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0],
  [1, 0, 0, 0],
  [0, 0, 1, 0],
  [0, 1, 0, 0],
  [0, 0, 0, 1],
  [0, 1, 0, 0],
  [1, 0, 0, 0]],
 '3': [[0, 1, 0, 0],
  [0, 0

In [67]:
for node in lkl_dict:
    npmm_sum=0
    for pos_list in lkl_dict[node]:
        npmm=np.matmul(pos_list,[0.25,0.25,0.25,0.25])
        npmm_sum+=math.log(npmm)
    print("Node ",node,"npmm: ",npmm_sum)

### Liklihood of Node 6 is the liklihod of observing the tree

Node  1 npmm:  -41.588830833596695
Node  2 npmm:  -41.588830833596695
Node  3 npmm:  -41.588830833596695
Node  4 npmm:  -41.588830833596695
Node  5 npmm:  -41.588830833596695
Node  9 npmm:  -75.38590510870945
Node  8 npmm:  -124.33743778967978
Node  7 npmm:  -68.08960098928449
Node  6 npmm:  -182.05582421991764


Ignore the code below!!

In [49]:
#for obj_with_seq in objects_with_sequence:
for obj_with_seq in nodes_with_sequence:
    # Perform operations or access attributes for objects without sequences
    print(f"Object without sequence: {obj_with_seq.name, obj_with_seq.parent, obj_with_seq.child}")


lkl_dict.keys()

Object without sequence: ('1', '9', '1')
Object without sequence: ('2', '9', '2')
Object without sequence: ('3', '8', '3')
Object without sequence: ('4', '7', '4')
Object without sequence: ('5', '7', '5')


dict_keys(['1', '2', '3', '4', '5', '9', '8', '7', '6'])

In [52]:
children = dict()
for node in Tree_Data:
    if node.parent not in children:
        children[node.parent] = [node.child]
    else:
        children[node.parent].append(node.child)

print(children)
for parent, childs in children.items():
    print(childs)

{'9': ['1', '2'], '8': ['9', '3'], '7': ['4', '5'], '6': ['7', '8']}
['1', '2']
['9', '3']
['4', '5']
['7', '8']


In [ ]:
# Separating external and internal nodes
external_nodes = []
internal_nodes = []

for node in Tree_Data:
    if node.getSequence() is not None:
        external_nodes.append(node)
    else:
        internal_nodes.append(node)

# Loop through objects with sequences
for node in external_nodes:
    # Perform operations or access attributes for objects with sequences
    print(f"Object with sequence: {node.name}")
    

# Loop through objects without sequences
for node in internal_nodes:
    # Perform operations or access attributes for objects without sequences
    print(f"Object without sequence: {node.name}")